In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

pd.options.display.max_columns=20

In [4]:
base = pd.read_csv("baseE&PFinal.csv")

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(base.drop(['Status'], axis=1), base['Status'], test_size=0.3)

In [11]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1090 entries, 2092 to 992
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   MW                 1090 non-null   float64
 1   DILI               1090 non-null   float64
 2   Cyto- toxicity     1090 non-null   float64
 3   HLM                1090 non-null   float64
 4   Cyp1A2 Inhibitor   1090 non-null   float64
 5   Cyp3A4 Inhibitor   1090 non-null   float64
 6   Cyp2D6 Inhibitor   1090 non-null   float64
 7   Cyp2C9 Inhibitor   1090 non-null   float64
 8   Cyp2C19 Inhibitor  1090 non-null   float64
 9   BBB                1090 non-null   float64
 10  P-gp Inhibitor     1090 non-null   float64
 11  P-gp Substrate     1090 non-null   float64
 12  hERG Blocker       1090 non-null   float64
 13  MMP                1090 non-null   float64
 14  AMES               1090 non-null   float64
 15  MRTD (mg/day)      1090 non-null   float64
dtypes: float64(16)
memory usage

KNN

In [6]:
#CROSS-VALIDAITION (Valor referência)

classifier = KNeighborsClassifier()
classifier.fit(base.drop(['Status'], axis=1), base["Status"])

scores = cross_val_score(classifier, base.drop(['Status'], axis=1), base["Status"], cv=5)
print(scores)
print(np.mean(scores))

[0.55433287 0.57634113 0.60385144 0.58953168 0.59504132]
0.5838196899594923


In [49]:
classifier = KNeighborsClassifier()

param_grid = {
    'n_neighbors': [2, 5, 10, 50, 60, 70, 100],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'weights': ['uniform'],
    'p': [3]
}

grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, cv=5, n_jobs=-2)
grid_search.fit(x_train, y_train)

scoreKNN = pd.DataFrame(grid_search.cv_results_)
scoreKNN.sort_values(by="mean_test_score", ascending=False, inplace=True)
print(scoreKNN[["mean_test_score", "param_metric", "param_n_neighbors", "param_weights"]])

print("--Best Classifier--")
print(f"Mean cv score: {grid_search.best_score_}")
y_pred = grid_search.best_estimator_.predict(x_test)
scoreBestKNN = accuracy_score(y_test, y_pred)
print(f"Score with unseen data: {scoreBestKNN}")
print(grid_search.best_params_)

    mean_test_score param_metric  param_n_neighbors param_weights
3          0.632715    euclidean                 50       uniform
11         0.632714    manhattan                 60       uniform
10         0.631932    manhattan                 50       uniform
17         0.630750    minkowski                 50       uniform
13         0.630365    manhattan                100       uniform
12         0.628395    manhattan                 70       uniform
20         0.627222    minkowski                100       uniform
6          0.625258    euclidean                100       uniform
19         0.623286    minkowski                 70       uniform
5          0.623285    euclidean                 70       uniform
4          0.617775    euclidean                 60       uniform
9          0.616993    manhattan                 10       uniform
18         0.615808    minkowski                 60       uniform
2          0.615416    euclidean                 10       uniform
16        

DWNN

In [7]:
#CROSS VALIDATION (Valor referência)

classifier = KNeighborsClassifier(weights='distance')
classifier.fit(base.drop(['Status'], axis=1), base["Status"])

scores = cross_val_score(classifier, base.drop(['Status'], axis=1), base["Status"], cv=5)
print(scores)
print(np.mean(scores))

[0.57634113 0.59009629 0.61898212 0.59366391 0.57575758]
0.5909682039855855


In [52]:
classifier = KNeighborsClassifier()

param_grid = {
    'n_neighbors': [25, 50, 100, 120, 150, 200, 210, 220, 250],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'weights': ['distance'],
    'p': [3]
}

grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, cv=5, n_jobs=-2)
grid_search.fit(x_train, y_train)

scoreDWNN = pd.DataFrame(grid_search.cv_results_)
scoreDWNN.sort_values(by="mean_test_score", ascending=False, inplace=True)
print(scoreDWNN[["mean_test_score", "param_metric", "param_n_neighbors", "param_weights"]])

print("--Best Classifier--")
print(f"Mean cv score: {grid_search.best_score_}")
y_pred = grid_search.best_estimator_.predict(x_test)
scoreBestDWNN = accuracy_score(y_test, y_pred)
print(f"Score with unseen data: {scoreBestDWNN}")
print(grid_search.best_params_)

    mean_test_score param_metric  param_n_neighbors param_weights
14         0.625646    manhattan                200      distance
15         0.621715    manhattan                210      distance
11         0.620930    manhattan                100      distance
12         0.620929    manhattan                120      distance
13         0.620928    manhattan                150      distance
16         0.620535    manhattan                220      distance
24         0.620136    minkowski                210      distance
26         0.619352    minkowski                250      distance
6          0.619348    euclidean                210      distance
20         0.618963    minkowski                100      distance
23         0.618958    minkowski                200      distance
21         0.618567    minkowski                120      distance
10         0.618567    manhattan                 50      distance
2          0.617782    euclidean                100      distance
25        

Logistic Regression (Regressão Logística)


In [8]:
#CROSS-VALIDAITION (Valor referência)

classifier = LogisticRegression()
classifier.fit(base.drop(['Status'], axis=1), base["Status"])

scores = cross_val_score(classifier, base.drop(['Status'], axis=1), base["Status"], cv=5)
print(scores)
print(np.mean(scores))

[0.5997249  0.59834938 0.65337001 0.61019284 0.58264463]
0.6088563514348184


In [77]:
classifier = LogisticRegression()

param_grid = {
    'penalty': ['l2'],
    'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
    'max_iter': [100, 200, 500, 1000, 2000],
    'C': [0.2, 0.4, 0.5, 0.6, 0.7, 1.0]
}

grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, cv=5, n_jobs=-2)
grid_search.fit(x_train, y_train)

scoreRL = pd.DataFrame(grid_search.cv_results_)
scoreRL.sort_values(by="mean_test_score", ascending=False, inplace=True)
print(scoreRL[["mean_test_score", "param_penalty", "param_solver", "param_max_iter", "param_C"]])

print("--Best Classifier--")
print(f"Mean cv score: {grid_search.best_score_}")
y_pred = grid_search.best_estimator_.predict(x_test)
scoreBestRL = accuracy_score(y_test, y_pred)
print(f"Score with unseen data: {scoreBestRL}")
print(grid_search.best_params_)

     mean_test_score param_penalty     param_solver  param_max_iter  param_C
79          0.599288            l2        liblinear            1000      0.5
73          0.599288            l2        liblinear             500      0.5
67          0.599288            l2        liblinear             200      0.5
85          0.599288            l2        liblinear            2000      0.5
61          0.599288            l2        liblinear             100      0.5
..               ...           ...              ...             ...      ...
106         0.597715            l2              sag             500      0.6
113         0.597715            l2             saga            1000      0.6
107         0.597715            l2             saga             500      0.6
112         0.597715            l2              sag            1000      0.6
111         0.597715            l2  newton-cholesky            1000      0.6

[180 rows x 5 columns]
--Best Classifier--
Mean cv score: 0.599288399362653

In [78]:
classifier = LogisticRegression()

param_grid = {
    'penalty': ['l2', 'l1'],
    'solver': ['liblinear'],
    'max_iter': [100, 200, 500, 1000],
    'C': [0.2, 0.4, 0.5, 0.6, 0.7, 1.0]
}

grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, cv=5, n_jobs=-2)
grid_search.fit(x_train, y_train)

scoreRL = pd.DataFrame(grid_search.cv_results_)
scoreRL.sort_values(by="mean_test_score", ascending=False, inplace=True)
print(scoreRL[["mean_test_score", "param_penalty", "param_solver", "param_max_iter", "param_C"]])

print("--Best Classifier--")
print(f"Mean cv score: {grid_search.best_score_}")
y_pred = grid_search.best_estimator_.predict(x_test)
scoreBestRL = accuracy_score(y_test, y_pred)
print(f"Score with unseen data: {scoreBestRL}")
print(grid_search.best_params_)

    mean_test_score param_penalty param_solver  param_max_iter  param_C
47         0.600075            l1    liblinear            1000      1.0
41         0.600075            l1    liblinear             100      1.0
45         0.600075            l1    liblinear             500      1.0
43         0.600075            l1    liblinear             200      1.0
22         0.599288            l2    liblinear            1000      0.5
16         0.599288            l2    liblinear             100      0.5
20         0.599288            l2    liblinear             500      0.5
18         0.599288            l2    liblinear             200      0.5
40         0.598895            l2    liblinear             100      1.0
42         0.598895            l2    liblinear             200      1.0
36         0.598895            l2    liblinear             500      0.7
34         0.598895            l2    liblinear             200      0.7
32         0.598895            l2    liblinear             100  

Árvore de Decisão

In [5]:
#CROSS-VALIDAITION (valor de referência)

classifier = DecisionTreeClassifier()
classifier.fit(base.drop(['Status'], axis=1), base["Status"])

scores = cross_val_score(classifier, base.drop(['Status'], axis=1), base["Status"], cv=5)
print(scores)
print(np.mean(scores))


[0.61623109 0.60247593 0.59697387 0.62121212 0.63636364]
0.6146513275811762


In [67]:
classifier = DecisionTreeClassifier()

param_grid = {
    'min_samples_leaf': [1, 2, 3, 5, 10, 50],
    'min_samples_split': [2, 4, 6, 10, 12, 20, 100],
    'max_depth': [None, 50, 100, 200, 500],
    'criterion': ['entropy', 'gini', 'log_loss'],
    'max_features': ['sqrt', 6, 16]
}

grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, cv=5, n_jobs=-2)
grid_search.fit(x_train, y_train)

scoreDT = pd.DataFrame(grid_search.cv_results_)
scoreDT.sort_values(by="mean_test_score", ascending=False, inplace=True)
print(scoreDT[["mean_test_score", "param_min_samples_leaf", "param_max_depth", "param_criterion", "param_min_samples_split", "param_max_features"]])

print("--Best Classifier--")
print(f"Mean cv score: {grid_search.best_score_}")
y_pred = grid_search.best_estimator_.predict(x_test)
scoreBestDT = accuracy_score(y_test, y_pred)
print(f"Score with unseen data: {scoreBestDT}")
print(grid_search.best_params_)

      mean_test_score  param_min_samples_leaf param_max_depth param_criterion  \
1889         0.693273                      50             500        log_loss   
246          0.693273                      50              50         entropy   
120          0.693273                      50            None         entropy   
119          0.693273                      50            None         entropy   
503          0.693273                      50             200         entropy   
...               ...                     ...             ...             ...   
435          0.611086                       3             200         entropy   
1051         0.611086                       1             200            gini   
679          0.610317                       2            None            gini   
1806         0.610304                       1             500        log_loss   
883          0.606761                       1             100            gini   

      param_min_samples_spl

Random Forest (Floresta Aleatória)

In [9]:
#CROSS-VALIDAITION (valor referência)

classifier = RandomForestClassifier()
classifier.fit(base.drop(['Status'], axis=1), base["Status"])

scores = cross_val_score(classifier, base.drop(['Status'], axis=1), base["Status"], cv=5)
print(scores)
print(np.mean(scores))

[0.63823934 0.69463549 0.68088033 0.6969697  0.684573  ]
0.6790595715817673


In [68]:
classifier = RandomForestClassifier()

param_grid = {
    'n_estimators': [20, 50, 100, 250, 500, 1000],
    'criterion': ['log_loss', 'gini', 'entropy'],
    'max_features': ['sqrt', 6, 16],
    'max_depth': [None, 100, 250, 500]
}

grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, cv=5, n_jobs=-2)
grid_search.fit(x_train, y_train)

scoreRF = pd.DataFrame(grid_search.cv_results_)
scoreRF.sort_values(by="mean_test_score", ascending=False, inplace=True)
print(scoreRF[["mean_test_score", "param_n_estimators", "param_criterion", "param_max_features", "param_max_depth"]])

print("--Best Classifier--")
print(f"Mean cv score: {grid_search.best_score_}")
y_pred = grid_search.best_estimator_.predict(x_test)
scoreBestRF = accuracy_score(y_test, y_pred)
print(f"Score with unseen data: {scoreBestRF}")
print(grid_search.best_params_)

     mean_test_score  param_n_estimators param_criterion param_max_features  \
75          0.710965                 250            gini               sqrt   
167         0.709782                1000         entropy               sqrt   
131         0.709393                1000            gini               sqrt   
112         0.709392                 500            gini               sqrt   
147         0.709000                 250         entropy               sqrt   
..               ...                 ...             ...                ...   
68          0.679897                 100        log_loss                 16   
198         0.679111                  20         entropy               sqrt   
156         0.677155                  20         entropy                 16   
48          0.675973                  20        log_loss                 16   
210         0.672828                  20         entropy                 16   

    param_max_depth  
75             None  
167    

SVM

In [10]:
#CROSS VALIDATION (Valor referência)

classifier = SVC()
classifier.fit(base.drop(['Status'], axis=1), base["Status"])

scores = cross_val_score(classifier, base.drop(['Status'], axis=1), base["Status"], cv=5)
print(scores)
print(np.mean(scores))

[0.58046768 0.59009629 0.64786795 0.58815427 0.58815427]
0.5989480903823783


In [82]:
classifier = SVC()

param_grid = {
    'kernel': ['linear', 'rbf'],
    'C': [0.2, 0.4, 0.5, 0.7, 0.9, 1.0, 2.0],
    'gamma': [0.05, 0.1, 0.3, 0.5, 1.0, 2.0]
}

grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, cv=5, n_jobs=-2)
grid_search.fit(x_train, y_train)

scoreSVM = pd.DataFrame(grid_search.cv_results_)
scoreSVM.sort_values(by="mean_test_score", ascending=False, inplace=True)
print(scoreSVM[["mean_test_score", "param_kernel", "param_C", "param_gamma"]])

print("--Best Classifier--")
print(f"Mean cv score: {grid_search.best_score_}")
y_pred = grid_search.best_estimator_.predict(x_test)
scoreBestSVM = accuracy_score(y_test, y_pred)
print(f"Score with unseen data: {scoreBestSVM}")
print(grid_search.best_params_)

    mean_test_score param_kernel  param_C  param_gamma
19         0.610308          rbf      0.4         0.50
77         0.609919          rbf      2.0         0.30
45         0.609917          rbf      0.7         1.00
69         0.609520          rbf      1.0         1.00
57         0.608734          rbf      0.9         1.00
..              ...          ...      ...          ...
6          0.582378       linear      0.2         0.50
4          0.582378       linear      0.2         0.30
2          0.582378       linear      0.2         0.10
0          0.582378       linear      0.2         0.05
11         0.575703          rbf      0.2         2.00

[84 rows x 4 columns]
--Best Classifier--
Mean cv score: 0.6103081540151292
Score with unseen data: 0.5954128440366973
{'C': 0.4, 'gamma': 0.5, 'kernel': 'rbf'}


Avaliação do melhor modelo

In [ ]:
#Avaliação do modelo com melhores hiper parâmetros

classifier = RandomForestClassifier(n_estimators=250, criterion='gini', max_features='sqrt', max_depth=None)
classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_test)
simple_score = accuracy_score(y_test, y_pred)

from sklearn.metrics import confusion_matrix
val = confusion_matrix(y_test, y_pred)

print(simple_score)
print(val)

print((val[0][0])/(val[0][0]+val[0][1]))
print((val[1][1])/(val[1][0]+val[1][1]))
print(x_test.shape)

0.6844036697247706
[[334 171]
 [173 412]]
0.6613861386138614
0.7042735042735043
(1090, 16)
